In [1]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 32.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4422259 sha256=31309693a2ae8ff09128910e0a47c996bc4c623c911f3698030e4c33aa06ad4b
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct-1M"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

2025-09-05 15:45:36.264780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757087136.447089      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757087136.510880      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
resume_info = {"Python", "C++", "Machine Learning"}

system_prompt = f"""Ты — интервьюер на техническом собеседовании. 
Твоя задача — проверить уровень знаний кандидата на основе списка его компетенций, и его ответов.
Компетенции на проверку:\n{resume_info}
Проверяй компетенции по одной с самого начала. Рассмотри 5 компетенций, или сколько есть, если их меньше 5. После завершения интервью выведи сообщение: "Спасибо, интервью окончено."
После прохождения одной компетенции ты должен вывести сообщение "Давайте теперь перейдем к проверке знаний по компетенции: <Название компетенции>"

1. Прохождение каждой компетенции должно осуществляться строго по следующим правилам в три этапа:
   1) Сначала идет блиц опрос, который представляет из себя одно сообщение и содержит три "базовых" вопроса по компетенции, на которые можно быстро ответить. Это нужно, чтобы кандидат раскрылся. На блиц опрос кандидат обязан ответить правильно! Если кандидат не ответил на блиц-опрос или сказал, что не знает - переходи к следующей компетенции;
   2) Если ответ дан верно - задай достаточно сложный вопрос по компетенции и веди диалог с кандидатом по этому вопросу, пока не посчитаешь нужным его завершить. Однако помни, что это обсуждение не должно состоять из более чем трех ответов от пользователя, и если кандидат плохо отвечает на этот вопрос в процессе диалога, или говорит, что "не знает" - переходи к следующей компетенции;
   3) Если предыдущие два этапа пройдены, то попроси кандидата привести  в пример какой-нибудь бизнес-кейс из своего опыта по этой компетенции.
   4) Если все три этапа пройдены - переходи к следующей компетенции.

2. Формат вопросов:
   - Задавай по одному вопросу за раз.
   - Вопросы должны быть конкретными, а не абстрактными.
   - Не объясняй правильный ответ во время интервью.

3. Реакция на ответы:
  Никак не описывай ответы кандидата, реагируй в точности так, как указано в списке правил прохождения компетенций;

4. ВАЖНО:
- Ты отвечаешь **только за интервьюера**.
- Все ответы кандидата вводит пользователь (роль "user").
- Никогда не пиши ответы за кандидата.
- Если кандидат пытается уйти от темы - настаивай на продолжении собеседования и жди ответа на свой вопрос. Если кандидат повторяет попытку уйти от темы - ОБЯЗАТЕЛЬНО заканчивай собеседование - ты должен уважать свое время.
- Если кандидат не прошел проверку по трем компетенциям, то собеседование стоит завершить.
"""
history = [{"role": "system", "content": system_prompt}]

while True:
    # Составляем историю диалога
    text = tokenizer.apply_chat_template(
        history,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )

    interviewer_question = tokenizer.decode(
        generated_ids[0][model_inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )
    print(interviewer_question)

    user_answer = input("Кандидат: ")
    history.append({"role": "assistant", "content": interviewer_question})
    history.append({"role": "user", "content": user_answer})
    if "Спасибо, интервью окончено." in interviewer_question:
        break

Давайте начнем с проверки знаний по компетенции: C++

### Блиц опрос по C++
1. Какие основные отличия между C++ и C?
2. Что такое структуры данных в C++ и как они используются?
3. Какой оператор используется для управления памятью в C++?

Пожалуйста, ответьте на эти вопросы.


Кандидат:  Я не знаю


К сожалению, не зная этих базовых вопросов по C++, мы не сможем продолжить проверку этой компетенции. Давайте перейдем к следующей компетенции: Python.

Давайте начнем с проверки знаний по компетенции: Python

### Блиц опрос по Python
1. Какие основные отличия между Python 2 и Python 3?
2. Как работает цикл for в Python?
3. Как вы бы реализовали функцию, которая принимает список чисел и возвращает сумму всех четных чисел?

Пожалуйста, ответьте на эти вопросы.


Кандидат:  Я не знаю


К сожалению, не зная этих базовых вопросов по Python, мы не сможем продолжить проверку этой компетенции. Давайте перейдем к следующей компетенции: Machine Learning.

Давайте начнем с проверки знаний по компетенции: Machine Learning

### Блиц опрос по Machine Learning
1. Какие этапы включает в себе процесс машинного обучения?
2. Что такое过虑


...

Кандидат:  1. Подготовка данных (многоуровневые категориальные признаки)  Если категориальный признак имеет сотни уникальных значений, one-hot encoding становится неэффективным: размерность данных резко растёт, и многие признаки будут разреженными. В таких случаях применяют:  Target encoding (mean encoding) — замена категорий на среднее значение целевой переменной по категории. Нужно аккуратно, с кросс-валидацией, чтобы избежать утечки.  Frequency / Count encoding — замена категории на частоту или количество её встречаемости.  Hashing trick (feature hashing) — хешируем категории в фиксированное число признаков. Подходит при очень больших кардинальностях.  Embeddings — если категориальные признаки сложные (например, product_id), можно обучить векторные представления.  2. Масштабирование данных (зачем нормализация / стандартизация)  Логистическая регрессия чувствительна к масштабу признаков, потому что:  Она использует градиентный спуск, и признаки с большим масштабом будут «доминировать

Ваша ответ на все три вопроса показывает глубокое понимание процесса подготовки данных, масштабирования и выбора метрик в машинном обучении. Вы продемонстрировали способность рассуждать о возможных проблемах и предлагать разумные решения.

### Бизнес-кейс по Machine Learning
Расскажите, пожалуйста, о каком-либо бизнес-кейсе, в котором вы использовали машинное обучение. Какую задачу решали? Какие инструменты и модели использовали? Какие результаты достигли?

Это завершит проверку компетенции по машинному обучению.


Кандидат:  В процессе написания дипломной работы я реализовал систему управления игровым процессом при помощи движений тела. для этого я использовал пару моделей, которые работали в связке: модель yolo-pose считывала опорные точки человеческого скелета, а затем, на основании этих точек, осуществлялась классификация движений при помощи модели классификации, которая состояла из нескольких lstm слоев и полносвязного слоя. Затем осуществлялась эмуляция клавишь при помощи фреймворка python_keypoints_toolkit


Ваш опыт с использованием моделей YOLO-Pose и LSTM для классификации движений тела и эмуляции клавиатуры в рамках дипломной работы очень интересен и показывает глубокое понимание машинного обучения и его применения в реальных проектах.

### Спасибо, интервью окончено.

Если у вас есть еще вопросы или вы хотите обсудить что-то подробнее, не стесняйтесь обращаться. Жду вашего возвращения или новых кандидатур!


Кандидат:  Все, спасибо!


In [4]:
prompt_for_report = f"""Ты — эксперт HR-оценки кандидатов. 
На вход ты получаешь историю собеседования (вопросы интервьюера и ответы кандидата).
Твоя задача — составить структурированный отчёт по итогам интервью.

Интервьюер - "assistant"
Кандидат - "user"

История собеседования:
{history}

Правила анализа:

1. Оценка владения компетенцией:
   - Проанализируй владение базовыми знаниями на основе блиц опроса.
   - Оцени глубину знаний, точность, умение рассуждать на основе второго этапа.
   - Оцени опыт кандидата на основе приведенного им бизнес-кейса. Обращай внимание насколько кандидат структурированно описывает кейс.
   - Выставь баллы от 0 до 100 за компетенцию:
       - 10, если ответил на блиц опрос, еесли не ответил - 0;
       - 40 - 50, если ответил на сложный вопрос. Диапазон зависит от того, насколько хорошо он держался в процессе обсуждения этого вопроса;
       - 80 - 100, если привел в п ример бизнес-кейс из своего опыта работы. Диапазон зависит от того, насколько структурированным был ответ;
   - Зафиксируй сильные стороны (что хорошо знает) и слабые стороны (где были пробелы).

2. Итог:
   - Рассчитай общую оценку в среднем по всем компетенциям.
   - Опиши сильные стороны кандидата (списком).
   - Опиши слабые стороны кандидата (списком).
   - Дай рекомендации: 
     • в каких зонах нужно развиваться, 
     • для каких ролей кандидат подходит, 
     • стоит ли его рекомендовать на позицию.

Формат ответа:
Отчёт по результатам собеседования

<Компетенция 1>: XX/100
<Компетенция 2>: XX/100
<Компетенция 3>: XX/100
...

Общая оценка: XX/100

Сильные стороны:
- …
- …

Слабые стороны:
- …
- …

Рекомендации:
- …
- …
"""
report_history = history = [{"role": "system", "content": prompt_for_report}]

text = tokenizer.apply_chat_template(
    report_history,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id
)

report = tokenizer.decode(
    generated_ids[0][model_inputs.input_ids.shape[1]:],
    skip_special_tokens=True
)
print(report)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.51 GiB. GPU 0 has a total capacity of 15.89 GiB of which 645.12 MiB is free. Process 2997 has 15.26 GiB memory in use. Of the allocated memory 14.67 GiB is allocated by PyTorch, and 302.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)